# Problem 3

This problem involves writing a program which performs an A-orthogonalization process on an input SPD matrix $A$.  We'll start with standard Gram-Schmidt routine, but modify it so the returned matrix has columns which are all A-orthogonal (conjugate) to each other.  This is one of the things the conjugate gradient algorithm does (under the covers) when solving the linear system $Ax = b$.

The algorithm proceeds as follows:

1. Take as input an $N x N$ matrix $A$.  We take the columns of A to be a set of column vectors $u_k$: $$A = \begin{pmatrix}\vdots & \vdots & & \vdots \\ u_1 & u_2 & \dots & u_n \\ \vdots & \vdots & & \vdots \end{pmatrix}$$The goal is to use the space spanned by the columns of $A$ to create a set of vectors $d_k$ which also span the space of $A$, and additionally satisfy $d_j^TAd_i=0$ for $i \neq j$, and $d_j^TAd_i=C_i$ for $i=j$ where $C_i$ is some constant.

2. Find the first vector $d_1=u_1$

3. Find the second vector $d_2=u_2−\beta_{21}d_1$ where $\beta_{21}=\frac{u_2^TAd_1}{d_1^TAd_1}$. Note that this removes the part of $u_2$ which is pointing in the same conjugate direction as $d_1$, leaving $d_2$ A-orthogonal to $d_1$. **Please prove this is the case for $d_2$ using pencil and paper, and hand in your proof along with your program.** $$proj_{d_1}u_2= \frac{d_1 \cdot u_2}{d_1 \cdot d_1}d_1 \rightarrow \frac{d_1A^Tu_2}{d_1A^Td_1}d_1$$<br> According to Shewchuk, two vectors $d_i$ and $d_j$ are conjugate if $d_i^TAd_j = 0$. If $u_2$ and $d_1$ are already A-orthogonal, there is no change to $u_2$; otherwise, the projection is subtracted, leaving $d_2$ to be the A-orthogonal component of $u_2$ to $d_1$

4. Find the remaining $d_k$ vectors using the iteration $$ d_k = u_k - \sum_{j=1}^{k-1}{\beta_{kj}d_j} $$

5. Assemble the result matrix and return it. $$D=\begin{pmatrix} \vdots & \vdots & \vdots & & \vdots \\ d_1 & d_2 & d_3 & \dots & d_N \\ \vdots & \vdots & \vdots & & \vdots \end{pmatrix}$$

Please make sure you include a test function which verifies that all columns in your D matrix are A-orthogonal to each other. Keep in mind that CG works for symmetric positive-definite (SPD) matrices, so you only need to test your algo against SPD inputs.  Also note that testing is tricky because the Gram-Schmidt process is not very stable numerically.  Therefore, it can accumulate large round-off errors. Feel free to use a generous testing tolerance to verify your results




In [3]:
from sklearn.datasets import make_spd_matrix
import numpy as np

def conjugate_gram_schmidt(A):
    # 1. The goal is to use the space spanned by the columns of 𝐴 to create a set
    # of vectors 𝑑𝑘 which also span the space of 𝐴
    D = np.zeros(A.shape)

    # 2. Find the first vector 𝑑1=𝑢1
    D[:,0] = A[:,0]

    # 3/4. Find the remaining  𝑑𝑘  vectors using the iteration
    for i in range(1, A.shape[0]):
        D[:, i] = A[:,i]
        for k in range(i):
            Bik = (- A[:,i].T @ A @ D[:,k]) / (D[:,k].T @ A @ D[:,k])
            D[:,i] += Bik * D[:,k]

    # 5. Assemble the result matrix and return it.
    return D

for N in range(5, 20):
    A = make_spd_matrix(N)
    D = conjugate_gram_schmidt(A)

    # The goal is to satisfy dj^TAdi=0 for i != j, and dj^TAdi=Ci for i=j
    C = []
    failed = False
    for i in range(D.shape[0]):
        for j in range(D.shape[1]):
            res = D[:,j].T @ A @ D[:, i]
            if i == j:
                C.append(res)
            else:
                if abs(res) > 1e-10:
                    print(f"\n\nFAILED AT N={N} WITH ERROR {res}\n\n")
                    failed = True
                    break
    if not failed:
        print(f"PASSED AT N={N} WITH CONSTANTS {C}")

PASSED AT N=5 WITH CONSTANTS [35.07200693436962, 0.5229535958644551, 1.728515250955378, 0.39868504107357, 0.030577436898111137]
PASSED AT N=6 WITH CONSTANTS [38.71847946051632, 0.49941470815004685, 0.13209020827605505, 0.026727013746950277, 0.0766939102785124, 0.056457144193562465]
PASSED AT N=7 WITH CONSTANTS [62.789872867356166, 1.687611852433794, 0.32321456926912123, 0.06297379269179942, 0.024684163277588777, 0.010464489192718208, 0.005327541507304973]
PASSED AT N=8 WITH CONSTANTS [248.51585548706618, 0.15984408113227755, 0.08796281090167589, 0.06528318429554056, 0.05659202218351499, 0.01575013969486874, 0.002507952189126362, 0.00305967637555223]
PASSED AT N=9 WITH CONSTANTS [7.870333090698721, 0.2713052883407118, 0.5483115717940206, 0.20588634394481722, 0.28185315286556883, 0.11529452919896029, 0.009179573582781991, 0.027436193742334233, 0.025715133761052996]
PASSED AT N=10 WITH CONSTANTS [125.92126471541826, 0.4699716876851798, 0.48740291845393485, 0.12582658083387596, 0.373292046